In [1]:
%pylab notebook
from modules import sim_mf_maps as sim
#from modules import moment_fits_v0 as gmf
from modules import moment_fits_v2 as gmf
import healpy as h

ioff()

Populating the interactive namespace from numpy and matplotlib


# Simulating multi-frequency MBB maps

In [17]:
# Setting the MBB simulation parameters here.
nside=8
npix=h.nside2npix(nside)
nu_min=10        # GHz
nu_max=3000      # GHz
num_channels=50
T1=9.75 ; dT1=4.
#T2=T1 ; dT2=dT1
T2=15.7 ; dT2=4.
alpha1=1.63 ; dalpha1=0.1
alpha2=alpha1 ; dalpha2=dalpha1
alpha2=2.82 ; dalpha2=0.1
N=3 ; dN=1.
figpath="./figures/sim_vary-T-N10-alpha_wide/"
! mkdir $figpath

pname=["Nside","nu_min","nu_max","Channels","T1","dT1","alpha1","dalpha1","T2","dT2","alpha2","dalpha2","N","dN"]
pval=[nside,nu_min,nu_max,num_channels,T1,dT1,alpha1,dalpha1,T2,dT2,alpha2,dalpha2,float(N),float(dN)]
param=open(figpath + "param.txt","w")
for i in range(size(pname)):
    param.write('%-10s %5.2f\n' % (pname[i], pval[i]))
param.close()

mkdir: ./figures/sim_vary-T-N10-alpha_wide/: File exists


In [18]:
o=sim.sim_methods(npix,nu_min,nu_max,num_channels)
o.gen_mf_data(T1,dT1,T2,dT1,alpha1,dalpha1,alpha2,dalpha2,N,dN)
o.normalize_data(1e7,2000.)
#o.gen_data_plots(figpath)

# Passing multi-frequency map data to the global moment fitting functions

In [19]:
gf=gmf.mbb_moment_fits(npix,o.nu,o.data,figpath=figpath)
gf.get_data_monopole()

# Fitting the monopole with moments upto 3rd order

In [20]:
gf.fit_mbb_monopole(gf.nu,gf.monopole,ub=[50,9,15],maxfev=50000)

T = 17.063735332  alpha= 2.97489053051  A= 0.000446667014397


# Carrying out the fitting at 1,2 and 3 order

In [21]:
gf.fit_mbb_order1(prange=1000.)
gf.fit_mbb_order2(prange=1000.)
gf.fit_mbb_order3(prange=1000.,maxfev=500000)

This took: 0.002312 seconds per pixel.
This took: 0.0051969296875 seconds per pixel.
This took: 1.17272585026 seconds per pixel.


ion()
figure() ; sl=3.
par=gf.par_monopole
#plot(gf.nu,gf.monopole/(gf.nu**sl),"k-",lw=2,label="Monopole")
#plot(gf.nu,gf.mbb_moments(gf.nu,T=par[0],alpha=par[1],A=par[2],p22=par[3],p23=par[4],p33=par[5],p222=par[6],p223=par[7],p233=par[8],p333=par[9])/(gf.nu**sl),"b--",lw=2,label="MBB fit")
#plot(gf.nu,abs(gf.monopole-gf.mbb_moments(gf.nu,T=par[0],alpha=par[1],A=par[2],p22=par[3],p23=par[4],p33=par[5],p222=par[6],p223=par[7],p233=par[8],p333=par[9]))/gf.monopole,"r--",lw=2,label="Difference")
#loglog()